In [2]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy
from sentistrength import PySentiStr
from nltk.util import trigrams
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"
from threading import Thread
import queue
import time
from csv import writer

In [2]:
from AllenNLPModel import useAllen

error loading _jsonnet (this is expected on Windows), treating C:\Users\Le\AppData\Local\Temp\tmpss25nx4n\config.json as plain json
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Le\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

nlp = spacy.load("en_core_web_sm")
def cleanData(messages):
    for i in range(0,len(messages)):
        try:
            doc = nlp(messages.iloc[i])
            token_list = [token for token in doc]
            filtered_tokens = [token for token in doc if not token.is_stop]
            lemmas = [token.lemma_ for token in filtered_tokens]
            s = ' '.join(lemmas)
            
            messages.iloc[i] = re.sub(r'\W+', ' ', s)
        except:
            messages.iloc[i] = ""
        
        #if is_ascii(messages.iloc[i]) == False:
            # messages.iloc[i] = ""

    return messages

In [5]:
senti = PySentiStr()
senti.setSentiStrengthPath('C:/Users/Le/Documents/Thesis/Code/SentiStrength/SentiStrength.jar')
senti.setSentiStrengthLanguageFolderPath('C:/Users/Le/Documents/Thesis/Code/SentiStrength/SentiStrength_Data')

def useSentiStrength(message):
    try:
        result = np.sum(senti.getSentiment(message.split()))
        result = result/np.sqrt(result**2 + 15)
    except: 
        result = 0
    
    return  result

sia = SentimentIntensityAnalyzer()
def useNLTK(message):
    try:
        result = sia.polarity_scores(message)['compound']
    except: 
        result == 0
      
    return result


# Single Thread Sentiment

In [ ]:
#Load Data
#data = pd.read_csv("GawaherClean.csv", index_col=0)
data = pd.read_csv("CleanData/GawaherClean.csv", index_col=0)

#Get thread
threads = data["ThreadID"].unique()
thread = data[data["ThreadID"]== threads[0]]
thread = thread.reset_index(drop = True)

#get all messages
messages = thread["Message"].copy()

clean = cleanData(messages.copy())
print(len(clean))

In [ ]:
sentiSentiment = []
nltkSentiment = []
allenSentiment = []

counter = 0
for i in clean:
    sentiSentiment.append(useSentiStrength(i))
    nltkSentiment.append(useNLTK(i))
    allenSentiment.append(useAllen(messages.iloc[counter]))
    counter += 1

In [ ]:
sentimentData = pd.DataFrame(np.array([messages.to_numpy(),sentiSentiment,nltkSentiment,allenSentiment]).T,
                             columns = ["Messages","SentiStrength","NLTK","Allen"])

sentimentData = sentimentData.sort_values(["Allen","SentiStrength","NLTK"])
sentimentData = sentimentData.dropna()
sentimentData = sentimentData.reset_index(drop=True)

In [ ]:
plt.figure(figsize = (12,6))
plt.hist(sentimentData["NLTK"], alpha = .5, label = "NLTK", bins = 25)
plt.hist(sentimentData["SentiStrength"], alpha = .5, label = "SentiStrength", bins = 25)
plt.hist(sentimentData["Allen"], alpha = .5, label = "Allen", bins = 25)
plt.legend();

In [ ]:
i = 1

test = messages.iloc[i]
print(test + "\n")

print("NLTK: " + str(useNLTK(test)))
print("SentiStrength: " + str(useSentiStrength(test)))
print("AllenNLP (Bert): " + str(useAllen(test)))

#find paper on AllenNLP and how it performs better than other models

# Get All Sentiments

In [6]:
pd.options.mode.chained_assignment = None

def getSentiments(row):
  
    message = row["Message"]
    row["AllenNLP"] = useAllen(message)
    
    message = cleanRow(message)
    row["NLTK"] = useNLTK(message)
    row["Sentistrength"] = useSentiStrength(message)
    
    with open('GawaherSentiment.csv', 'a', encoding="utf-8") as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(list(row))
        f_object.close()
    
def cleanRow(message):
    try:
        doc = nlp(message)
        token_list = [token for token in doc]
        filtered_tokens = [token for token in doc if not token.is_stop]
        lemmas = [token.lemma_ for token in filtered_tokens]
        s = ' '.join(lemmas)
            
        messages = re.sub(r'\W+', ' ', s)
    except:
        messages = ""
    
    return messages
    

In [7]:
data = pd.read_csv("CleanData/GawaherClean.csv", index_col=0)

data["NLTK"] = np.zeros(len(data))
data["Sentistrength"] = np.zeros(len(data))
data["AllenNLP"] = np.zeros(len(data))

In [3]:
test = pd.read_csv("GawaherSentiment.csv")


In [4]:
(len(test)

2403

In [ ]:
for i in range(0,len(data)):
    getSentiments(data.iloc[i])

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors
C:\Users\Le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sentistrength\__init__.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_text = pd.Series(df_text)


#rows = queue.Queue()

allThreads = []

for i in range(2,len(data)):
    row = Thread(target= getSentiments, args=(data,i))
    allThreads.append(row)
    
for thread in allThreads:
    thread.start()
    
for thread in allThreads:
    thread.join()

#rows.qsize()

# Thread Sentiments

In [ ]:
sia = SentimentIntensityAnalyzer()
def getAllSentiment(thread):
    messages = thread["Message"].copy()
    clean = cleanData(messages)
    sentiment = []
    for i in clean:
        sentiment.append(useNLTK(i))
    
    return np.mean(sentiment)

In [ ]:
#Load Data
#data = pd.read_csv("Ansar1Clean.csv", index_col=0)
data = pd.read_csv("CleanData/GawaherClean.csv", index_col=0)

#Get thread
threads = data["ThreadID"].unique()


In [ ]:
sentiments = []

for i in range(0,len(threads)):
    thread = data[data["ThreadID"]== threads[i]]
    sentiments.append(getAllSentiment(thread))

In [ ]:
plt.figure(figsize=(8,5))
plt.hist(sentiments, bins = 25)
plt.xlabel("Sentiment", fontsize = 14)
plt.ylabel("count", fontsize = 14)
plt.title("Sentiment Distribution of Cleaned Data", fontsize = 14)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14);

plt.savefig("Images/SentimentDistribution.png")

In [ ]:
np.mean(sentiments)

In [ ]:
np.median(sentiments)

In [ ]:
np.std(sentiments)

In [ ]:
sum(np.array(sentiments) < 0)/len(sentiments)

In [ ]:
sentimentData = pd.DataFrame(np.array([threads,sentiments]).T, columns = ["thread","sentiment"])

sentimentData.to_csv("sentimentData.csv", index = False)